In [1]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")

In [2]:
from AlphaNet.Data import DataLoader,generate_shift_data

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template
Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [3]:
# def generate_shift_data(alpha_name,shift,data_path = "/home/wuwenjun/Data_lib/ti0/wuwenjun/"):
#     dataloader = DataLoader()
#     dataloader.load_data_from_file(alpha_name = alpha_name, data_path=data_path,end_date="2022-01-01")
    
#     # generate shift list
#     sequence_list = [0]
#     for i in range(shift-1,30,shift):
#         sequence_list.append(i)
#     sequence_list.sort(reverse=True)
    
#     # shift
#     x = dataloader.feature_data.drop("target",axis=1)
#     final_df = []
#     for index,group in tqdm(x.groupby("ticker")):
#         group_df = []
#         for i in sequence_list:
#             a = group.shift(i)
#             a.columns = [str(j)+"_Shift_%i" % i for j in group.columns]
#             group_df.append(a)
#         group_df = pd.concat(group_df,axis=1)
#         final_df.append(group_df)
#     final_df = pd.concat(final_df,axis=0)
#     final_df = pd.concat([final_df,dataloader.feature_data["target"]],axis=1)
#     convert_to_standard_daily_data_par(df=final_df, output_name=alpha_name +"_Shift_" + shift, 
#                                        output_path="/home/wuwenjun/Data_lib/ti0/wuwenjun/")
#     return final_df

In [5]:
a = generate_shift_data("test",5)

100%|██████████| 4353/4353 [00:13<00:00, 324.50it/s]


In [9]:
a.dropna(axis=0,inplace=True)

test_mcorr_open_low_10_Shift_29  test_1_Shift_29  \
timestamp           ticker                                                     
2020-02-20 09:30:00 000001                         0.115025         0.115025   
                    000002                         0.796874         0.796874   
                    000004                         0.815852         0.815852   
                    000005                         0.815519         0.815519   
                    000006                         0.846951         0.846951   
...                                                     ...              ...   
2021-05-31 09:30:00 688698                         0.966120         0.966120   
                    688699                         0.938115         0.938115   
                    688777                         0.896094         0.896094   
                    688788                         0.912368         0.912368   
                    688981                         0.968343         0.968343   

                            test_Shift_29  ewmCorr(high,low)_Shift_29  \
timestamp           ticker                                              
2020-02-20 09:30:00 000001        3604.78                    0.414245   
                    000002        9428.61                    0.968666   
                    000004         184.01                   -0.086929   
                    000005          57.28                    0.656316   
                    000006         379.05                    0.967827   
...                                   ...                         ...   
2021-05-31 09:30:00 688698          37.55                    0.279355   
                    688699         174.40                    0.688512   
                    688777         165.50                   -0.030966   
                    688788         200.17                    0.419786   
                    688981         112.47                    0.918865   

                            ewmCorr(high,low,7)_Shift_29  \
timestamp           ticker                                 
2020-02-20 09:30:00 000001                      0.414245   
                    000002                      0.968666   
                    000004                     -0.086929   
                    000005                      0.656316   
                    000006                      0.967827   
...                                                  ...   
2021-05-31 09:30:00 688698                      0.279355   
                    688699                      0.688512   
                    688777                     -0.030966   
                    688788                      0.419786   
                    688981                      0.918865   

                            test_mcorr_open_low_10_Shift_24  test_1_Shift_24  \
timestamp           ticker                                                     
2020-02-20 09:30:00 000001                         0.910903         0.910903   
                    000002                         0.951134         0.951134   
                    000004                         0.778103         0.778103   
                    000005                         0.946824         0.946824   
                    000006                         0.942247         0.942247   
...                                                     ...              ...   
2021-05-31 09:30:00 688698                         0.619987         0.619987   
                    688699                         0.877278         0.877278   
                    688777                         0.777743         0.777743   
                    688788                         0.936346         0.936346   
                    688981                         0.892976         0.892976   

                            test_Shift_24  ewmCorr(high,low)_Shift_24  \
timestamp           ticker                                              
2020-02-20 09:30:00 000001        3674.64                    0.979625

In [10]:
a

test_mcorr_open_low_10_Shift_29  test_1_Shift_29  \
timestamp           ticker                                                     
2020-01-02 09:30:00 000001                              NaN              NaN   
                    000002                              NaN              NaN   
                    000004                              NaN              NaN   
                    000005                              NaN              NaN   
                    000006                              NaN              NaN   
...                                                     ...              ...   
2021-06-01 09:30:00 688699                         0.883972         0.883972   
                    688777                         0.857100         0.857100   
                    688788                         0.850705         0.850705   
                    688819                              NaN              NaN   
                    688981                         0.958937         0.958937   

                            test_Shift_29  ewmCorr(high,low)_Shift_29  \
timestamp           ticker                                              
2020-01-02 09:30:00 000001            NaN                         NaN   
                    000002            NaN                         NaN   
                    000004            NaN                         NaN   
                    000005            NaN                         NaN   
                    000006            NaN                         NaN   
...                                   ...                         ...   
2021-06-01 09:30:00 688699         173.55                    0.945779   
                    688777         167.65                    0.551271   
                    688788         197.62                    0.355893   
                    688819            NaN                         NaN   
                    688981         113.17                    0.897014   

                            ewmCorr(high,low,7)_Shift_29  \
timestamp           ticker                                 
2020-01-02 09:30:00 000001                           NaN   
                    000002                           NaN   
                    000004                           NaN   
                    000005                           NaN   
                    000006                           NaN   
...                                                  ...   
2021-06-01 09:30:00 688699                      0.945779   
                    688777                      0.551271   
                    688788                      0.355893   
                    688819                           NaN   
                    688981                      0.897014   

                            test_mcorr_open_low_10_Shift_24  test_1_Shift_24  \
timestamp           ticker                                                     
2020-01-02 09:30:00 000001                              NaN              NaN   
                    000002                              NaN              NaN   
                    000004                              NaN              NaN   
                    000005                              NaN              NaN   
                    000006                              NaN              NaN   
...                                                     ...              ...   
2021-06-01 09:30:00 688699                         0.916000         0.916000   
                    688777                         0.799730         0.799730   
                    688788                         0.942118         0.942118   
                    688819                         0.804634         0.804634   
                    688981                         0.886539         0.886539   

                            test_Shift_24  ewmCorr(high,low)_Shift_24  \
timestamp           ticker                                              
2020-01-02 09:30:00 000001            NaN                         NaN